In [1]:
%cd ..

/home/rinjac/ondewo/ondewo-t2s-stella


In [35]:
from notebooks.nemo_modules import InferenceDataLayer
import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.tts as nemo_tts
import wave
import numpy as np
from scipy.io.wavfile import write
import IPython

In [3]:
neural_factory = nemo.core.NeuralModuleFactory(placement=nemo.core.DeviceType.GPU, backend=nemo.core.Backend.PyTorch)

In [11]:
data_preprocessor = nemo_asr.AudioToMelSpectrogramPreprocessor.import_from_config(
        "notebooks/tacotron2.yaml", "AudioToMelSpectrogramPreprocessor"
    )

[NeMo I 2020-08-18 15:06:34 features:144] PADDING: 16
[NeMo I 2020-08-18 15:06:34 features:152] STFT using conv
[NeMo I 2020-08-18 15:06:34 neural_modules:363] Instantiated a new Neural Module of type `AudioToMelSpectrogramPreprocessor` using configuration loaded from the `notebooks/tacotron2.yaml` file


Create spectrogram

In [5]:
waveglow_conf = {
    'config-path': "models/waveglow/waveglow.yaml",
    'model-path': "models/waveglow/checkpoints_waveglow_kerstin/WaveGlowNM.pt",
    'sigma': 0.6
}

In [6]:
waveglow = nemo_tts.WaveGlowInferNM.import_from_config(
    waveglow_conf['config-path'], "WaveGlowInferNM",
    overwrite_params={"sigma": waveglow_conf['sigma']}
)
waveglow.restore_from(waveglow_conf['model-path'])

[NeMo I 2020-08-18 15:06:22 neural_modules:363] Instantiated a new Neural Module of type `WaveGlowInferNM` using configuration loaded from the `models/waveglow/waveglow.yaml` file


In [38]:
waveglow.setup_denoiser()

In [7]:
afile = wave.open('notebooks/sandra.wav')
samples = afile.getnframes()
audio = afile.readframes(samples)
audio_np_ = np.copy(np.frombuffer(audio, dtype=np.int16))
audio_np = audio_np_ / 2 ** 15
sample = audio_np

In [51]:
data_layer = InferenceDataLayer(sample=sample)
audio_signal, audio_signal_len = data_layer()
processed_signal, processed_signal_len = data_preprocessor(
    input_signal=audio_signal,
    length=audio_signal_len)
audio_pred = waveglow(mel_spectrogram=processed_signal)
audio_pred, mel_len_pred = neural_factory.infer(tensors=[audio_pred, processed_signal_len])

[NeMo I 2020-08-18 15:30:37 actions:734] Evaluating batch 0 out of 1


In [52]:
end_index: int = mel_len_pred[0].cpu().numpy()[0] * 256
audio = audio_pred[0].cpu().numpy()[0][:end_index]

In [53]:
audio, _ = waveglow.denoise(audio, strength=0.05)

In [54]:
save_file = "notebooks/sandra_generated_denoise.wav"

In [55]:
# conversion to 16-bit PCM
audio *= 32768
audio = audio.astype("int16")

# save audio to file
write(save_file, 22050, audio)

In [56]:
IPython.display.Audio(save_file)